<a href="https://colab.research.google.com/github/gmshroff/metaLearning2022/blob/main/code/nb0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TRAINING RECURRENT NEURAL NETWORKS

In [ ]:
# !pip install import_ipynb --quiet
# !git clone https://github.com/gmshroff/metaLearning.git
# %cd metaLearning/code

In [2]:
import import_ipynb
import utils
import models
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import pickle
import numpy as np

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from models.ipynb


In [ ]:
from course_data import MyDS, TsDS, FeedData

Financial Time Series

In [ ]:
%cd ../data

In [ ]:
!unzip ../data/feedDataTrain.pickle.zip

In [ ]:
%ls

In [ ]:
with open('./feedDataTrain.pickle','rb') as f: feedDataTrain=pickle.load(f)

In [ ]:
!unzip feedDataTest.pickle.zip

In [ ]:
with open('./feedDataTest.pickle','rb') as f: feedDataTest=pickle.load(f)

Recurrent network: LSTM

In [ ]:
net = models.SimpleLSTM(input_size=149,hidden_size=64,output_size=3,lr=1e-3)

In [ ]:
net,losses,accs=models.Train(net,feedDataTrain.train_ds,epochs=25,verbose=True)

In [ ]:
models.accuracy_variable(net,feedDataTrain.train_ds)

In [ ]:
models.accuracy_variable(net,feedDataTrain.test_ds)

In [ ]:
models.accuracy_variable(net,feedDataTest.train_ds)

In [ ]:
feedDataTrain.train_ds.samples[0].shape

Siusoidal time series data

In [ ]:
#Generate data - sinusoidal mix
train_ds, test_ds, dloader = utils.sinDataset(n_samples=100,length=20,n_classes=5,batch_size=1)

Recurrent network: RNN

In [ ]:
# Define an RNN to process a sequence; default is 1-D where input can be (batch,seq_vals) else input will need to be given
# with an extra dimension and input will be (batch,seq,features)
net = models.SimpleRNN(input_size=1,hidden_size=32,output_size=5,lr=1e-3)

In [ ]:
# Train the network; note that network is trained in place so repeated calls further train it.
net,losses,accs=models.Train(net,dloader,epochs=50,verbose=True)

Training/Test Accuracies

In [ ]:
s_tr=torch.concat([s.unsqueeze(0).unsqueeze(0) for s in train_ds.samples])
l_tr=torch.concat([l.unsqueeze(0).unsqueeze(0) for l in train_ds.labels])

In [ ]:
newtr=MyDS(s_tr,l_tr)

In [ ]:
models.accuracy_variable(net,newtr)

In [ ]:
s_te=torch.concat([s.unsqueeze(0).unsqueeze(0) for s in test_ds.samples])
l_te=torch.concat([l.unsqueeze(0).unsqueeze(0) for l in test_ds.labels])

In [ ]:
newte=MyDS(s_te,l_te)

In [ ]:
models.accuracy_variable(net,newte)

Recurrent network: LSTM

In [ ]:
net = models.SimpleLSTM(input_size=1,hidden_size=32,output_size=5,lr=1e-3)

In [ ]:
# Train the network; note that network is trained in place so repeated calls further train it.
net,losses,accs=models.Train(net,dloader,epochs=50,verbose=True)

In [ ]:
models.accuracy_variable(net,newtr)

In [ ]:
models.accuracy_variable(net,newte)

Transformer (Encoder) on Financial Time Series

In [ ]:
net = models.Transformer(input_size=149,hidden_size=128,output_size=3,lr=1e-3,num_layers=3)

In [ ]:
net,losses,accs=models.Train(net,feedDataTrain.train_ds,epochs=10,verbose=True)

In [ ]:
models.accuracy_variable(net,feedDataTrain.train_ds)

In [ ]:
models.accuracy_variable(net,feedDataTrain.test_ds)

In [ ]:
models.accuracy_variable(net,feedDataTest.test_ds)

Transformer (Encoder) on Sinusoidal Time Series

In [ ]:
net = models.Transformer(input_size=1,hidden_size=32,output_size=5,lr=1e-3,num_layers=3)

In [ ]:
s_tr=torch.concat([s.unsqueeze(0).unsqueeze(0) for s in train_ds.samples])
l_tr=torch.concat([l.unsqueeze(0).unsqueeze(0) for l in train_ds.labels])

In [ ]:
newtr=MyDS(s_tr,l_tr)

In [ ]:
net,losses,accs=models.Train(net,newtr,epochs=20,verbose=True)

In [ ]:
models.accuracy_variable(net,newtr)

In [ ]:
s_te=torch.concat([s.unsqueeze(0).unsqueeze(0) for s in test_ds.samples])
l_te=torch.concat([l.unsqueeze(0).unsqueeze(0) for l in test_ds.labels])

In [ ]:
newte=MyDS(s_te,l_te)

In [ ]:
models.accuracy_variable(net,newte)

Sanity check

In [ ]:
correct=0
for s in newte:
    print(s[1].item(), torch.argmax(net(s[0])).numpy())
    if s[1].item()==torch.argmax(net(s[0])): correct+=1

In [ ]:
correct/len(newte)

Plots

In [ ]:
for s,l in zip(feedDataTrain.train_ds.samples,feedDataTrain.train_ds.labels):
    if l==2: plt.plot(s[0,:,1])
plt.show()

In [ ]:
for s,l in zip(train_ds.samples,train_ds.labels):
    if l==4: plt.plot(s)
plt.show()

ICL as GD

In [188]:
m,n,r=3,4,1
W=torch.randn(r,m,requires_grad=True)
X=torch.randn(m,n,requires_grad=True)
Y=torch.randn(r,n,requires_grad=True)

In [226]:
L=(1/(2*n))*(W@X-Y)@(W@X-Y).T

In [227]:
L

tensor([[4.1142]], grad_fn=<MmBackward0>)

In [228]:
DW=torch.autograd.grad(L,W)[0]

In [229]:
DW

tensor([[-3.2649,  3.5639,  1.2275]])

In [230]:
Z=torch.concat((X,Y))

In [231]:
def LSA(Z,WK,WQ,WV,P):
    return Z+P@WV@Z@((WK@Z).T)@(WQ@Z)

In [232]:
U=torch.concat((torch.concat((torch.eye(m,m),torch.zeros(m,r)),dim=1),torch.zeros(r,m+r)))

In [233]:
WK,WQ=U,U

In [234]:
WV=torch.concat((torch.zeros(m,m+r),torch.concat((W,-torch.eye(r,r)),dim=1)))

In [236]:
P=(1/n)*torch.eye(m+r,m+r)

In [237]:
Z_new=LSA(Z,WK,WQ,WV,P)

In [238]:
Z_new

tensor([[ 7.5657e-01, -1.1331e+00,  7.6973e-04,  2.4923e+00],
        [-1.2428e+00,  2.2424e+00, -7.2748e-01, -4.2421e-01],
        [ 9.8772e-01,  2.0951e+00, -3.7867e-01,  1.4858e+00],
        [-5.5284e+00,  1.4484e+01, -2.4025e+00, -7.4237e+00]],
       grad_fn=<AddBackward0>)

In [239]:
Y+DW@X

tensor([[-5.5284, 14.4836, -2.4025, -7.4237]], grad_fn=<AddBackward0>)

In [240]:
W_new=W-DW

In [241]:
W_new

tensor([[ 1.9820, -2.7947, -0.4314]], grad_fn=<SubBackward0>)

In [242]:
W

tensor([[-1.2829,  0.7693,  0.7961]], requires_grad=True)

In [243]:
L_new=L=(1/2*n)*(W_new@X-Y)@(W_new@X-Y).T

In [244]:
L_new

tensor([[280.6778]], grad_fn=<MmBackward0>)

In [246]:
Y_new=Z_new[-1,:]

In [247]:
L_new_hat=(1/2*n)*(W@X-Y_new)@(W@X-Y_new).T

In [248]:
L_new_hat

tensor([[280.6778]], grad_fn=<MmBackward0>)